In [1]:
# import tensorflow.contrib.eager as tfe



import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import keras
# import datetime as dt
from datetime import datetime
import time
# import datetime.datetime




from keras import layers
from keras import models
# from keras import optimizers
from tensorflow.keras import optimizers

from keras.applications.densenet import DenseNet121, DenseNet169
#from keras.applications.nasnet import NASNetLarge
# from keras_efficientnets import EfficientNetB7
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints
from keras.layers.normalization import BatchNormalization 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# from keras import backend as K
from tensorflow.keras import backend as K





tf.set_random_seed(1234)
random.seed(100)

os.environ["CUDA_VISIBLE_DEVICES"]="1"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)typ

In [2]:

def score_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    
def get_unique_classes(input_directory,files):

    unique_classes=set()
    for f in files:
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        unique_classes.add(c.strip())
    return sorted(unique_classes)

def one_hot_encoding(one_hot_vector,y, class2index):
    ind=class2index[y]
    one_hot_vector[ind]=1
    return one_hot_vector

# Search for multi-label subjects
def searching_overlap(input_directory,class2index, input_file_names):
    multiclasses=[]
    multisubjects=[]
    number = []
    for file in input_file_names:
        f=file
        g = f.replace('.mat','.hea')
        input_file = os.path.join(input_directory,g)
        with open(input_file,'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    if len(tmp)>1:
                        one_hot_vector = [0]*(len(class2index))
                        for c in tmp:
                            one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                        multiclasses.append(one_hot_vector)
                        multisubjects.append(g)
                        number.append(len(tmp))
    return multisubjects, multiclasses, number

def block_feature(sequence_en, minimum_len): 
    new_en = []
    if len(sequence_en) > minimum_len:  # 길이가 minimum보다 긴 경우
        start = random.randint(0,len(sequence_en)-minimum_len)
        #print(start)
        new_en = sequence_en[start:start+minimum_len]
    elif len(sequence_en) == minimum_len: # 길이가 minimum
        new_en = sequence_en
    else: 
        assert len(sequence_en) <= minimum_len
    return new_en

def exploratory_look(input_directory,file, class2index):
    classes = []
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                print(tmp, len(tmp))
    return tmp     

# Get classes of sorted file names
def get_labels(input_directory,file, class2index):
    f = file
    g = f.replace('.mat','.hea')
    input_file = os.path.join(input_directory,g)
    with open(input_file,'r') as f:
        for lines in f:
            if lines.startswith('#Dx'):
                tmp = lines.split(': ')[1].split(',')
                one_hot_vector = [0]*(len(class2index))
                for c in tmp:
                    one_hot_vector = one_hot_encoding(one_hot_vector, c.strip(), class2index)
                
    return one_hot_vector

def randextract_mels(curr_step, batch_size, data, mel_directory, class2index, minimum_len, x_mean_final, x_std_final):
    mel_files = []
    classes = []
    start = batch_size*curr_step
    end = batch_size*(curr_step+1)
    curr_file_indices = data[start:end]
    for file in curr_file_indices:
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        clip_file = block_feature(tmp_file, minimum_len)
        #print(clip_file.shape)
        #clip_file = tmp_file[:minimum_len]
        clip_file -= x_mean_final
        clip_file /= x_std_final
        mel_files.append(clip_file)
        label = get_labels(input_directory, file, class2index)
        classes.append(label)
    
    # YJS added for ABN -> should calculate 2 losses
#     classes_abn = [classes,classes]
    
    concat = list(zip(mel_files, classes))
#     concat = list(zip(mel_files, classes_abn))
    random.shuffle(concat)
    mel_files, classes = zip(*concat)
    return mel_files, classes

def train(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final): 
    loss=[]
#     acc = []
    f1 = []

    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps): # loops over batches
        batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)

        batch_mels = np.asarray(batch_mels)
        batch_labels = np.asarray(np.squeeze(batch_labels))
        
        batch_labels = [batch_labels, batch_labels]


# return of train
# 0 = total loss (attention branch + perception branch)
# 1 = loss of attention pred
# 2 = loss of perception pred
# 3 = f1 of attention pred
# 4 = f1 of perception pred     


        train_tmp = model.train_on_batch(batch_mels, batch_labels)
        loss_ = train_tmp[0]/2
        f1_ = np.mean((train_tmp[3], train_tmp[4]))
        loss.append(loss_)
        f1.append(f1_)

    loss = np.mean(np.array(loss))
    f1 = np.mean(np.array(f1))
    return loss, f1

def train_edit(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final): 
    loss=[]
#     acc = []
    f1 = []

    total_steps = int(np.ceil(len(data_train)/batch_size))
    for curr_step in range(total_steps): # loops over batches
        batch_mels, batch_labels = randextract_mels(curr_step, batch_size, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)

        batch_mels = np.asarray(batch_mels)
        batch_labels = np.asarray(np.squeeze(batch_labels))


        heatmap = CAM_conv1D(p_model, conv_layer, softmax_layer,  x_mean_final, x_std_final, minimum_len, 
                                  n_channels, batch_mels, batch_labels, out_len)        
        heatmap = np.asarray(heatmap)
        
        batch_labels = [batch_labels, batch_labels]


# return of train
# 0 = total loss (attention branch + perception branch)
# 1 = loss of attention pred
# 2 = loss of perception pred
# 3 = f1 of attention pred
# 4 = f1 of perception pred     

        train_tmp = model.train_on_batch([batch_mels, heatmap], batch_labels)
        loss_ = train_tmp[0]/2
        f1_ = np.mean((train_tmp[3], train_tmp[4]))
        loss.append(loss_)
        f1.append(f1_)

    loss = np.mean(np.array(loss))
    f1 = np.mean(np.array(f1))
    return loss, f1

def test(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    
    mel_files = []
    classes = []
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
#         gradcam_files = []
        heatmap_files=[]
        label = get_labels(input_directory, file, class2index)
        label = [label]
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final

            clip_file = clip_file.reshape(1,minimum_len,n_channels)


            mel_files.append(clip_file)    
#             gradcam_files.append(gradcam)

        mel_files = np.asarray(mel_files)

        mel_files = mel_files.reshape(steps,minimum_len,n_channels)

        logit = model.predict(mel_files)
        # YJS changed on 2020-06-02: input으로 두개 들어가야하니까 predict도 수정?
#         print(len(logit))
        logit = np.mean(logit, axis=0)
        logit = np.mean(logit, axis=0)
#         print(logit)
        pred = np.argmax(logit)
#         print('Pred={}'.format(pred))
        
        label = np.argmax(get_labels(input_directory, file, class2index))
#         print('Label={}'.format(label))
        #f1 = f1_score(label, logit)
        #print(pred, label)
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
        #total_f1 += f1
    final_acc = total_acc / i
    #final_f1 = total_f1 / i
    return final_acc#, final_f1

def test_edit(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    
    mel_files = []
    classes = []
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
#         gradcam_files = []
        heatmap_files=[]
        label = get_labels(input_directory, file, class2index)
        label = [label]
        for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
            start = block*minimum_len
            end = (block+1)*minimum_len
            clip_file = tmp_file[start:end]
            clip_file -= x_mean_final
            clip_file /= x_std_final
#             heatmap = CAM_conv1D(p_model, conv_layer, softmax_layer,  x_mean_final, x_std_final, minimum_len, 
#                                       n_channels, clip_file, label, out_len)
            
#             heatmap = grad_cam_conv1D(p_model, layer_nm, x_mean_final, x_std_final, minimum_len, n_channels, 
#                                      clip_file, label, out_len, sample_weight=1, keras_phase=0)
            clip_file = clip_file.reshape(1,minimum_len,n_channels)
            heatmap = CAM_conv1D(p_model, conv_layer, softmax_layer,  x_mean_final, x_std_final, minimum_len, 
                                      n_channels, clip_file, label, out_len)
#             print(len(heatmap))
#             print(len(heatmap[0]))
#             print(len(heatmap[0][0]))
#             print(len(heatmap[0][0][0]))
            mel_files.append(clip_file)    
#             gradcam_files.append(gradcam)
            heatmap_files.append(heatmap)
        mel_files = np.asarray(mel_files)
        heatmap_files = np.asarray(heatmap_files)
        
#         print(heatmap_files.shape)
        heatmap_files = heatmap_files.reshape(steps, out_len,1)
        mel_files = mel_files.reshape(steps,minimum_len,n_channels)
#         heatmap = np.asarray(heatmap)
#         gradcam_files = np.asarray(gradcam_files)
# grad_cam_conv1D(model, layer_nm,x_mean_final, x_std_final, minimum_len, n_channels, x, y,sample_weight=1,  keras_phase=0)        
        
# get_labels(input_directory,file, class2index)



#         logit = model.predict(mel_files)
#         logit = model.predict([mel_files, gradcam_files])
        logit = model.predict([mel_files, heatmap_files])
        # YJS changed on 2020-06-02: input으로 두개 들어가야하니까 predict도 수정?
#         print(len(logit))
        logit = np.mean(logit, axis=0)
        logit = np.mean(logit, axis=0)
#         print(logit)
        pred = np.argmax(logit)
#         print('Pred={}'.format(pred))
        
        label = np.argmax(get_labels(input_directory, file, class2index))
#         print('Label={}'.format(label))
        #f1 = f1_score(label, logit)
        #print(pred, label)
        if pred == label:
            acc = 1
        else:
            acc = 0
        total_acc += acc
        #total_f1 += f1
    final_acc = total_acc / i
    #final_f1 = total_f1 / i
    return final_acc#, final_f1


def test_short(data, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final):
    scores = []
    predicted_labels=[]
    accuracy=np.zeros(len(data))
    #total_loss=[]
    total_acc = 0
    total_f1 = 0
    
    mel_files = []
    classes = []
    for i, file in enumerate(data):
        tmp_file = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
        steps = int(np.floor(tmp_file.shape[0]/minimum_len))
        mel_files = []
#         gradcam_files = []
        heatmap_files=[]
        label = get_labels(input_directory, file, class2index)
        label = [label]
#         for block in range(steps): # 128개씩 쪼갠 블럭 단위로 predict
        steps=1
        block = 0
        start = block*minimum_len
        end = (block+1)*minimum_len
        clip_file = tmp_file[start:end]
        clip_file -= x_mean_final
        clip_file /= x_std_final

        clip_file = clip_file.reshape(1,minimum_len,n_channels)
#         heatmap = CAM_conv1D(p_model, conv_layer, softmax_layer,  x_mean_final, x_std_final, minimum_len, 
#                                       n_channels, clip_file, label, out_len)

        mel_files.append(clip_file)    
#         heatmap_files.append(heatmap)
        mel_files = np.asarray(mel_files)
#         heatmap_files = np.asarray(heatmap_files)
#         heatmap_files = heatmap_files.reshape(steps,out_len,1) # changed for modified attention editting
        mel_files = mel_files.reshape(steps,minimum_len,n_channels)

        logit = model.predict([mel_files, heatmap_files])
        # YJS changed on 2020-06-02: input으로 두개 들어가야하니까 predict도 수정?

        
        logit = np.mean(logit, axis=0)
        logit = np.mean(logit, axis=0)
        pred = np.argmax(logit)
        
        label = np.argmax(get_labels(input_directory, file, class2index))
        if pred == label:
            acc = 1
        else:
            acc = 0

        total_acc += acc

    final_acc = total_acc / i

    return final_acc#, final_f1
batch_size = 32#16#20#32#5#2#1#10#32
minimum_len = 2880
epochs = 1000
loss_function = 'binary_crossentropy' #'categorical_crossentropy'
activation_function = 'sigmoid' # 'softmax'
rootdir = '../'
date = datetime.today().strftime("%Y%m%d")
input_directory = os.path.join(rootdir, 'Training_WFDB')
mel_name = 'Raw_data_20200424' 
mel_directory = os.path.join(rootdir, mel_name)
# results_directory = os.path.join(rootdir, 'results_'+date+'_0_IEEE_n=1')
results_directory = os.path.join(rootdir, 'results_'+date+'_editting_CAM_multiclass')
if not os.path.isdir(input_directory):
    os.mkdir(input_directory)
if not os.path.isdir(mel_directory):
    os.mkdir(mel_directory)
if not os.path.isdir(results_directory):
    os.mkdir(results_directory)
        
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)
input_file_names = sorted(input_files)

unique_classes = get_unique_classes(input_directory, input_files)
class2index = {}
for a, b in enumerate(unique_classes):
    class2index[b] = a
    
# classes_orig= [x.replace('.mat', '.hea') for x in input_file_names] # total subjects
# classes_multi, _, _ = searching_overlap(input_directory,class2index, input_file_names)
# classes_single = [x for x in classes_orig if x not in classes_multi]
# classes_single = [x.replace('.hea', '.mat') for x in classes_single]
# classes_orig = 

# double-checking if classes_single have single-label
# a, b, c  = searching_overlap(input_directory,class2index,classes_single)

# we can safely use classes_single as input_file_names
# input_file_names = classes_single
random.shuffle(input_file_names)
np.shape(input_file_names)

x_mean_all = []
x_std_all = []
for file in input_file_names:
    x = np.load(mel_directory + '/' + file.replace('.mat', '.npy'))
    x_mean = [np.mean(x[:,0]), np.mean(x[:,1]), np.mean(x[:,2]), np.mean(x[:,3]), np.mean(x[:,4]), np.mean(x[:,5]),
             np.mean(x[:,6]), np.mean(x[:,7]), np.mean(x[:,8]), np.mean(x[:,9]), np.mean(x[:,10]), np.mean(x[:,11])]
    
    x_std = [np.std(x[:,0]), np.std(x[:,1]), np.std(x[:,2]), np.std(x[:,3]), np.std(x[:,4]), np.std(x[:,5]),
             np.std(x[:,6]), np.std(x[:,7]), np.std(x[:,8]), np.std(x[:,9]), np.std(x[:,10]), np.std(x[:,11])]
    #print(x_mean)
    x_mean_all.append(x_mean)
    x_std_all.append(x_std) # yjs corrected on 2020-05-25
x_mean_final = np.mean(x_mean_all, axis=0)
x_std_final = np.mean(x_std_all, axis=0)
print(x_mean_final)

data, data_test = train_test_split(input_file_names, test_size = 0.2, train_size = 0.8, shuffle=True)
data_train, data_val = train_test_split(data, test_size = 0.05, train_size = 0.95, shuffle=True)
print(np.shape(data_train), np.shape(data_val), np.shape(data_test))


main_input = Input(shape=(minimum_len,12), dtype='float32', name='main_input')

branch_pred = []

[-0.0012702   0.00038759  0.00402317 -0.0029567   0.00051957  0.00116427
 -0.0008405  -0.00222516 -0.0019119  -0.00093502  0.00129789  0.00177598]
(5225,) (276,) (1376,)



In [18]:
classes_orig= [x.replace('.mat', '.hea') for x in input_file_names] # total subjects
classes_multi, _, _ = searching_overlap(input_directory,class2index, input_file_names)
classes_single = [x for x in classes_orig if x not in classes_multi]
classes_single = [x.replace('.hea', '.mat') for x in classes_single]

In [19]:
classes_multi

['A4459.hea',
 'A5188.hea',
 'A0182.hea',
 'A5051.hea',
 'A6327.hea',
 'A6807.hea',
 'A2514.hea',
 'A3895.hea',
 'A6474.hea',
 'A1027.hea',
 'A2392.hea',
 'A3620.hea',
 'A3678.hea',
 'A5190.hea',
 'A5795.hea',
 'A0224.hea',
 'A1870.hea',
 'A6232.hea',
 'A2357.hea',
 'A6360.hea',
 'A6250.hea',
 'A2027.hea',
 'A6211.hea',
 'A4075.hea',
 'A2278.hea',
 'A1043.hea',
 'A1689.hea',
 'A1164.hea',
 'A3140.hea',
 'A5441.hea',
 'A0881.hea',
 'A5412.hea',
 'A1344.hea',
 'A3681.hea',
 'A2652.hea',
 'A2078.hea',
 'A2653.hea',
 'A1326.hea',
 'A2553.hea',
 'A3955.hea',
 'A2503.hea',
 'A4716.hea',
 'A4993.hea',
 'A3512.hea',
 'A0607.hea',
 'A5971.hea',
 'A4648.hea',
 'A4536.hea',
 'A3009.hea',
 'A4178.hea',
 'A5794.hea',
 'A4992.hea',
 'A6234.hea',
 'A5181.hea',
 'A6394.hea',
 'A5488.hea',
 'A0575.hea',
 'A0835.hea',
 'A4408.hea',
 'A4352.hea',
 'A4223.hea',
 'A4829.hea',
 'A4306.hea',
 'A2330.hea',
 'A1108.hea',
 'A3231.hea',
 'A2878.hea',
 'A2282.hea',
 'A2572.hea',
 'A2898.hea',
 'A3491.hea',
 'A613

In [22]:
multilabels=[]
for file in classes_multi:
    label = get_labels(input_directory, file, class2index)
    multilabels.append(label)
multilabels

[[1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 0, 0],
 [1, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0,

In [23]:
len(multilabels)

476

In [24]:
class2index

{'AF': 0,
 'I-AVB': 1,
 'LBBB': 2,
 'Normal': 3,
 'PAC': 4,
 'PVC': 5,
 'RBBB': 6,
 'STD': 7,
 'STE': 8}

In [26]:
print(classes_multi[0])
print(multilabels[0])

A4459.hea
[1, 0, 0, 0, 0, 0, 1, 0, 0]


In [3]:
# val_acc = test_short(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)


### Attention editting by CAM

In [4]:
# curr_x = batch_mels[0]
# curr_x = curr_x.reshape(1,minimum_len,n_channels)
# print(curr_x.shape)
# softmax_layer = 'dense_final'
# softmax_out = p_model.get_layer(softmax_layer)
# softmax_weights = softmax_out.weights

# conv_layer = 'batch_normalization_12'
# conv_out = p_model.get_layer(conv_layer).output
# get_conv_out = K.function(p_model.input, [conv_out, softmax_weights[0]])
# conv_out, softmax_weights = get_conv_out(curr_x)

In [5]:

# Batch-wise to reduce time

CAMdir = '/home/taejoon/PhysioNetChallenge/results_20200601_1_CAM_primitive_model'
bestmodel = 'ECG_ABN_E87L0.33'
p_model = tf.keras.models.load_model(os.path.join(CAMdir, bestmodel), custom_objects={'score_f1' : score_f1}) # primitive model
conv_layer = 'batch_normalization_12'
softmax_layer = 'dense_final'
n_channels=12
out_len=12#128
softmax_weights = p_model.get_layer('dense_final').weights[0]
# model.get_layer(softmax_layer).weights[0]
def CAM_conv1D(model, conv_layer, softmax_layer, x_mean_final, x_std_final, minimum_len, 
                    n_channels, x, y, out_len):
    
    # x랑 y는 batch size만큼의 리스트 (32)
    heatmaps=[]    
    
    curr_x = np.asarray(x)
    curr_x = curr_x.reshape(len(x),minimum_len,n_channels)
    get_conv_out = K.function(model.input, [model.get_layer(conv_layer).output, model.get_layer(softmax_layer).weights[0]])
    conv_out, softmax_weights = get_conv_out(curr_x)

    for i in range(len(x)):

        curr_classes = y[i]
        class_index=[]
        [class_index.append(j) for j in range(len(curr_classes)) if curr_classes[j]==1]
        heatmap=np.zeros((1,36)) # might need to fix this if GradCAM or primitive model changes

        conv_out_ = conv_out[i] # (36, 128)


        for label in class_index:  # multiclass일 경우 대비해서 for문
            curr_weights = softmax_weights[:,label]
            weighted_conv = conv_out_*curr_weights
            
            weighted_conv = weighted_conv.sum(axis=-1) # output = (1,36)
            heatmap += weighted_conv
            

        heatmap %= len(class_index) # 단일 class일 경우 1로 나눠짐. 두개일 경우 더해진 heatmap들이 2로 나눠짐
#         heatmap = np.resize(heatmap, (1,out_len))
        heatmap = np.resize(heatmap, (out_len, 1))
        heatmaps.append(heatmap)
        
    return heatmaps

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:

# heatmap = CAM_conv1D(p_model, conv_layer, softmax_layer,  x_mean_final, x_std_final, minimum_len, 
#                                   n_channels, batch_mels, batch_labels, out_len)


In [7]:
# heatmap[0]

In [8]:
# len(heatmap)

In [9]:
# len(heatmap[0][0])

In [10]:
# len(heatmap[0][0][0]) # 32, 1, 128

In [11]:
# curr_x, _ = randextract_mels(0,32, data_train, mel_directory, class2index, minimum_len, x_mean_final, x_std_final)
# curr_x = np.asarray(curr_x)
# curr_x.shape
# curr_x = curr_x.reshape(32,minimum_len,12)

# # curr_x = curr_x[0]
# # curr_x = curr_x.reshape(1,minimum_len,12)
# CAMdir = '/home/taejoon/PhysioNetChallenge/results_20200601_1_CAM_primitive_model'
# bestmodel = 'ECG_ABN_E87L0.33'
# p_model = tf.keras.models.load_model(os.path.join(CAMdir, bestmodel), custom_objects={'score_f1' : score_f1}) # primitive model
# conv_layer = 'batch_normalization_12'
# softmax_layer = 'dense_final'
# n_channels=12
# out_len=128
# softmax_weights = p_model.get_layer('dense_final').weights[0]

# get_conv_out = K.function(p_model.input, p_model.get_layer(conv_layer).output)

# # conv_out = get_conv_out(curr_x)

In [12]:
# curr_x.shape

In [13]:
# conv_out = get_conv_out(curr_x)
# conv_out.shape

In [14]:
# conv_out[0].shape

In [15]:
# CAMdir = '/home/taejoon/PhysioNetChallenge/results_20200601_1_CAM_primitive_model'
# bestmodel = 'ECG_ABN_E87L0.33'
# p_model = tf.keras.models.load_model(os.path.join(CAMdir, bestmodel), custom_objects={'score_f1' : score_f1}) # primitive model
# conv_layer = 'batch_normalization_12'
# softmax_layer = 'dense_final'
# n_channels=12
# out_len=128
# def CAM_conv1D(model, conv_layer, softmax_layer, x_mean_final, x_std_final, minimum_len, 
#                     n_channels, x, y, out_len):
    
#     # x랑 y는 batch size만큼의 리스트 (32)
       
#     #레이어 이름에 해당되는 레이어 정보를 가져옴 
# #     conv_out = model.get_layer(conv_layer).output
# #     print(conv_out)
# #     conv_output= conv_out.output
# #     softmax_out = model.get_layer(softmax_layer)
# #     softmax_weights = softmax_out.weights
#     heatmaps=[]    
#     for file in range(len(x)): # 굳이 이렇게 batch 따로돌리는 이유는 나중에 multilable일 때 heatmap 두개일 경우를 대비하기위해
#         # 일단은 single label로 하기 
#         class_index=[]
#         curr_x  = x[file]
#         curr_x = curr_x.reshape(1,minimum_len,n_channels)
#         curr_classes = y[file]
#         [class_index.append(i) for i in range(len(curr_classes)) if curr_classes[i]==1]
#         heatmap=np.zeros((1,36)) # might need to fix this if GradCAM or primitive model changes
# #         curr_x = tf.convert_to_tensor(curr_x)
#         get_conv_out = K.function(model.input, [model.get_layer(conv_layer).output, model.get_layer(softmax_layer).weights[0]])
#         conv_out, softmax_weights = get_conv_out(curr_x)
#         print(file)
#         for label in class_index:  # multiclass일 경우 대비해서 for문
#             curr_weights = softmax_weights[:,label]
#             weighted_conv = conv_out*curr_weights
#             weighted_conv = weighted_conv.sum(axis=2) # output = (1,36)            
#             heatmap += weighted_conv
            
#         heatmap %= len(class_index) # 단일 class일 경우 1로 나눠짐. 두개일 경우 더해진 heatmap들이 2로 나눠짐
#         heatmap = np.resize(heatmap, (1,out_len))
# #         heatmap = cv2.resize(heatmap, (1,out_len))
#         heatmaps.append(heatmap)
        
#     return heatmaps

### Attention editting by GradCAM


In [16]:
# GradCAMdir = '/home/taejoon/PhysioNetChallenge/results_20200601_1_CAM_primitive_model'
# bestmodel = 'ECG_ABN_E87L0.33'
# p_model = tf.keras.models.load_model(os.path.join(GradCAMdir, bestmodel), custom_objects={'score_f1' : score_f1}) # primitive model
# layer_nm = 'conv1d_12'
# n_channels=12
# out_len=128
# def grad_cam_conv1D(model, layer_nm,x_mean_final, x_std_final, minimum_len, 
#                     n_channels, x, y, out_len,sample_weight=1,  keras_phase=0):
    
#     # x랑 y는 batch size만큼의 리스트 (32)
       
#     #레이어 이름에 해당되는 레이어 정보를 가져옴 
#     layers_wt = model.get_layer(layer_nm).weights
#     layers = model.get_layer(layer_nm)
#     layers_weights = model.get_layer(layer_nm).get_weights()

#     heatmaps=[]    
#     for file in range(len(x)): # 굳이 이렇게 batch 따로돌리는 이유는 나중에 multilable일 때 heatmap 두개일 경우를 대비하기위해
#         # 일단은 single label로 하기 
#         print(file)
#         class_index=[]
#         curr_x  = x[file]
# #         print(curr_x.shape)
#         curr_x = curr_x.reshape(1,minimum_len,n_channels)
#         curr_classes = y[file]
# #         print(curr_classes)
# #         print(curr_classes.shape)
#         [class_index.append(i) for i in range(len(curr_classes)) if curr_classes[i]==1]
#         heatmap=np.zeros(36) # might need to fix this if GradCAM or primitive model changes
#         for label in class_index:  # multiclass일 경우 대비해서 for문

#             #긍정 클래스를 설명할 수 있게 컨볼루션 필터 가중치의 gradient를 구함  
#             grads = K.gradients(model.output[:,label], layers_wt)[0]

#             #필터별로 가중치를 구함 
#             pooled_grads = K.mean(grads, axis=(0,1))
#             get_pooled_grads = K.function(model.input, 
#                                  [pooled_grads, layers.output[0]])

#             pooled_grads_value, conv_layer_output_value = get_pooled_grads(curr_x)
# #             print(conv_layer_output_value.shape) # (36,128)
#             for i in range(conv_layer_output_value.shape[-1]):
#                 conv_layer_output_value[:, i] *= pooled_grads_value[i]
#                 for j in range(len(conv_layer_output_value[:,i])):
#                     conv_layer_output_value[j,i] = max(0, conv_layer_output_value[j,i])
#                 #YJS manually added RELU function on 2020-06-02
# #                 conv_layer_output_value[:,i] = max(0, conv_layer_output_value[:,i])
#             heatmap += np.mean(conv_layer_output_value, axis=-1)
# #         print(class_index)
#         heatmap %= len(class_index) # 단일 class일 경우 1로 나눠짐. 두개일 경우 더해진 heatmap들이 2로 나눠짐
#         heatmap = cv2.resize(heatmap, (1,out_len))
#         heatmaps.append(heatmap)
# #     heatmaps = heatmaps.reshape(len(x),1,36)
#     return heatmaps

In [17]:
from ABNmodules_multiclass import *

# def get_custom_model(input_shape, n_classes, minimum_len, target_classes, out_ch=256, n=18):
# model = get_custom_model((None, 12), 9, minimum_len, 1, out_ch=256, n=1)
# model = get_model((None, 12), 9, n=7)

# model = cam_model((None, 12), 9, minimum_len, out_ch=256, n=18)


model = edit_model((None,12), len(unique_classes), minimum_len)
# model = get_custom_model((None, 12), 9, minimum_len, 1, out_ch=256, n=1)
model.summary()
model.compile(loss=loss_function,
              optimizer=optimizers.Adam(lr=1e-5),           
              metrics=[score_f1])



Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image (InputLayer)        [(None, None, 12)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 64)     2368        input_image[0][0]                
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, 64)     256         conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 64)     12352       batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
# # 중단된 training 이어돌리기위해 임시로 사용
# results_directory = results_directory.replace("0608", "0604")
# results_directory

In [ ]:
# # 중단된 training 이어돌리기 위해 임시로 사용
# latest = tf.train.latest_checkpoint(results_directory)
# latest
# model.load_weights(latest)

In [ ]:
# DISP DATETIME FOR CHECKING TIME
start = time.time()
val_acc_sum=[]
train_loss_sum=[]
train_acc_sum=[]
val_loss_sum=[]
val_acc_min = 0
print(results_directory)

for num_epoch in range(epochs):
#     num_epoch += 32 # 중단된 코드 돌리기 위해 임의로 사용
    random.shuffle(data_train)
    train_loss, train_f1 = train_edit(data_train, mel_directory, batch_size, class2index, minimum_len, x_mean_final, x_std_final)
    print('\nEpoch',num_epoch+1,'train_loss:',f'{train_loss:.3f}','train_f1:',f'{train_f1:.3f}',"\t")
    model_output = "ECG_ABN_E%02dL%.2f" % (num_epoch, train_loss)
    save_name = os.path.join(results_directory, model_output)
    
    val_acc = test_edit(data_val, mel_directory, input_directory, class2index, minimum_len, model, x_mean_final, x_std_final)

    if val_acc > val_acc_min:
        val_acc_min = val_acc
        model.save_weights(save_name.format(epoch=0))
#         model.save(save_name)
#         cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_name,
#                                                          save_weights_only=True,
#                                                          verbose=1)
    print('\nValidation', num_epoch+1, 'valid_acc:',f'{val_acc:.3f}', 'best_acc:',f'{val_acc_min:.3f}', "\t")
print(time.time() - start)

In [ ]:
# batch_mels =np.asarray(batch_mels)
# batch_mels = batch_mels.reshape(32,2880,12)
# heatmap = np.asarray(heatmap)
# heatmap = heatmap.reshape(batch_size, 1, out_len)
# a = model2.predict([batch_mels, heatmap])
# a